<a href="https://colab.research.google.com/github/OdysseusPolymetis/stanza_greek_tagger/blob/main/greek_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<center>**Lemmatisation, postagging et features pour le grec**</center>

Ce script permet d'analyser des fichiers en grec qui sont soit des **CSV** soit des fichiers **TXT**. Les fichiers **DOIVENT être encodés en UTF8**.

In [1]:
#Exécuter la cellule (avec le signe play sur le côté)
#Ne pas changer : c'est la partie où google télécharge stanza.

!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 574 kB 12.8 MB/s 
     |████████████████████████████████| 175 kB 52.6 MB/s 
     |████████████████████████████████| 4.2 MB 50.6 MB/s 
     |████████████████████████████████| 596 kB 37.0 MB/s 
     |████████████████████████████████| 86 kB 3.0 MB/s 
     |████████████████████████████████| 6.6 MB 44.4 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=e0f595b13aa8601ba27f0fa84bd2c6ee31ea157b5d7254e599d5f841de4e1fe5
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [17]:
#Bien attendre que la cellule précédente ait fini de s'exécuter. On le voit quand le signe play ne bouge plus ou que le logo google
#sur l'onglet du navigateur redevient orange (et non gris)

import sys
sys.path.insert(0,'/content')

C'est dans la cellule qui suit qu'on va déterminer le fichier à analyser. Ce fichier doit se trouver sur un google drive et être disponible pour tout utilisateur possédant le lien de partage.
<br><br>Quand on a le lien, il faut identifier l'identifiant du fichier, ou "id". Cet "id" c'est généralement ce qui exclut à la fois `drive.google.com/file/d/` et `/view?usp=sharing`. Exemple, pour un fichier `https://drive.google.com/file/d/1GgB1A1pT1IUL8uaoReF91OZvhLdcdz9p/view?usp=sharing`, l'"id" est `1GgB1A1pT1IUL8uaoReF91OZvhLdcdz9p`.
<br><br>Dans la cellule qui suit, vous devrez donc prendre votre "id", et le coller exactement à la ligne `id="1P1eZnW75XBhdYk9EoVjV-kCRU4-1_8aH"` à la place de l'identifiant déjà présent, sans enlever `id=` ou les guillemets (l'identifiant doit être entre guillemets).
<br><br>Il faut aussi, en fonction du fichier que vous souhaitez analyser, préciser l'extension (CSV ou TXT). Il faut donc changer la variable `extension` dans la cellule qui suit, en conservant bien les guillemets. Exemple : `extension="txt"`

In [33]:
#Exécuter cette cellule

import gdown
import os

if os.path.exists('./data/')==False:
  os.mkdir('./data/')
prefix="https://drive.google.com/uc?id="
id="1P1eZnW75XBhdYk9EoVjV-kCRU4-1_8aH"
url=prefix+id
extension="txt"
output='./data/text_to_analyze.'+extension
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1P1eZnW75XBhdYk9EoVjV-kCRU4-1_8aH
To: /content/data/text_to_analyze.txt
100%|██████████| 6.06M/6.06M [00:00<00:00, 194MB/s]


'./data/text_to_analyze.txt'

In [34]:
#Exécuter cette cellule

import glob
import stanza
from pathlib import Path
import csv
import pandas as pd

In [35]:
#Exécuter cette cellule

nlp_grc = stanza.Pipeline('grc', processors='tokenize,lemma,pos,depparse')

2022-06-14 11:14:08 INFO: Loading these models for language: grc (Ancient_Greek):
| Processor | Package |
-----------------------
| tokenize  | proiel  |
| pos       | proiel  |
| lemma     | proiel  |
| depparse  | proiel  |

2022-06-14 11:14:08 INFO: Use device: cpu
2022-06-14 11:14:08 INFO: Loading: tokenize
2022-06-14 11:14:08 INFO: Loading: pos
2022-06-14 11:14:08 INFO: Loading: lemma
2022-06-14 11:14:08 INFO: Loading: depparse
2022-06-14 11:14:09 INFO: Done loading processors!


#**Partie à exécuter seulement si on a un CSV de base**

Pour que cela fonctionne, vous devez avoir un **CSV** en **UTF8**, séparé par des **tabulations**.
<br>Le résultat des analyses sera dans un dossier `results` visible sur le côté du google colab.
<br><br>La variable `column_name` doit avoir la valeur du nom de votre colonne CSV qui contient les formes grecques. Ne pas oublier de bien conserver les guillemets. Changer comme suit, exemple : `column_name="text_form"` si la colonne a pour titre text_form.

In [ ]:
column_name="word"

In [21]:
files= glob.iglob('/content/data/**/*.csv', recursive=True)
headers=['Word','Lemma','POS','Features']
for filename in files :
    path=Path(filename).stem
    if os.path.exists('./results/')==False:
        os.mkdir('./results/')
    with open('./results/'+path+'.csv', 'w', newline='') as outcsv:
        number_occurrences=0
        writer = csv.writer(outcsv)
        writer.writerow(headers)
        print(filename)
        df = pd.read_csv(filename, sep='\t')
        words = df[column_name].tolist()
        print("Il y a "+str(len(words))+" formes à analyser.")
        gktxt = " ".join(words)
        grc_analyzed = nlp_grc(gktxt)
        for sent in grc_analyzed.sentences:
          for word in sent.words:
            row=list()
            row.append(word.text)
            row.append(word.lemma)
            row.append(word.upos)
            row.append(word.feats)
            writer.writerow(row)

/content/data/PHAT_VAT_GRC__RNN_1-10000.csv
Il y a 10000 formes à analyser.
Formes analysées : 1000/10000
Formes analysées : 2000/10000
Formes analysées : 3000/10000
Formes analysées : 4000/10000
Formes analysées : 5000/10000
Formes analysées : 6000/10000
Formes analysées : 7000/10000
Formes analysées : 8000/10000
Formes analysées : 9000/10000
Formes analysées : 10000/10000


#**Partie à exécuter seulement si le fichier de base est un TXT**

Pour que cela fonctionne, vous devez avoir un **TXT** en **UTF8**.
<br>Le résultat des analyses sera dans un dossier `results` visible sur le côté du google colab.

In [36]:
files= glob.iglob('/content/data/**/*.txt', recursive=True)
headers=['Word','Lemma','POS','Features']
for filename in files :
    path=Path(filename).stem
    if os.path.exists('./results/')==False:
        os.mkdir('./results/')
    with open('./results/'+path+'.csv', 'w', newline='') as outcsv:
        writer = csv.writer(outcsv)
        writer.writerow(headers)
        print(filename)
        gktxt=open(filename, encoding="utf8").read()
        grc_analyzed = nlp_grc(gktxt)
        for sent in grc_analyzed.sentences:
          for word in sent.words:
            row=list()
            row.append(word.text)
            row.append(word.lemma)
            row.append(word.upos)
            row.append(word.feats)
            writer.writerow(row)

/content/data/text_to_analyze.txt
